In [1]:
import pandas as pd
from requests import get

url = 'https://en.wikipedia.org/wiki/Areas_of_Chennai'
response = get(url)

In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
place_each = html_soup.find('table',class_='wikitable')
places = place_each.find_all('a')

list_of_places = []

for place in places:
    list_of_places.append(place.text)
    

In [4]:
from geopy import Nominatim
import numpy as np

In [5]:
                                ### extracting latitude and longitude from address ###

latitude = []
longitude = []

for address in list_of_places:
    print(address)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    if location is not None:
        lat = location.latitude
        long = location.longitude
        latitude.append(lat)
        longitude.append(long)
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

Adambakkam
Adyar
Alandur
Alapakkam
Alwarpet
Alwarthirunagar
Ambattur
Aminjikarai
Anna Nagar
Annanur
Arumbakkam
Ashok Nagar
Avadi
Ayanavaram
Besant Nagar
Basin Bridge
Chepauk
Chetput
Chintadripet
Chitlapakkam
Choolai
Choolaimedu
Chrompet
Egmore
Ekkaduthangal
Eranavur
Ennore
Foreshore Estate
Fort St. George
George Town
Gopalapuram
Government Estate
Guindy
Gerugambakkam
IIT Madras
Injambakkam
ICF
Iyyapanthangal
Jafferkhanpet
Karapakkam
Kattivakkam
Kattupakkam
Kazhipattur
K.K. Nagar
Keelkattalai
Kattivakkam
Kilpauk
Kodambakkam
Kodungaiyur
Kolathur
Korattur
Korukkupet
Kottivakkam
Kotturpuram
Kottur
Kovilambakkam
Koyambedu
Kundrathur
Madhavaram
Madhavaram Milk Colony
Madipakkam
Madambakkam
Maduravoyal
Manali
Manali New Town
Manapakkam
Mandaveli
Mangadu
Mannady
Mathur
Medavakkam
Meenambakkam
MGR Nagar
Minjur
Mogappair
MKB Nagar
Mount Road
Moolakadai
Moulivakkam
Mugalivakkam
Mudichur
Mylapore
Nandanam
Nanganallur
Nanmangalam
Neelankarai
Nemilichery
Nesapakkam
Nolambur
Noombal
Nungambakkam
Otte

In [6]:
location = geolocator.geocode('Chennai')
lat = location.latitude
long = location.longitude

In [7]:
dict_ = {'Places':list_of_places,'Latitude':latitude,'Longitude':longitude}
df = pd.DataFrame(dict_)
df = df.dropna()
df = df.reset_index(drop=True)

In [8]:
                                    ### defining foursquare credentials ###

CLIENT_ID =  # your Foursquare ID(hidden for uploading)
CLIENT_SECRET =  # your Foursquare Secret(hidden for uploading)
VERSION = '20180605' # Foursquare API version
LIMIT = 30
radius = 500

In [9]:
import requests
import json
from pandas.io.json import json_normalize

In [10]:
                        ### function for returning venues in given location coordinates ###
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        try:
            print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items'] #extracting the necessary from json file

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            #for loop ends here

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list]) #double for loop
            nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
        except Exception as e:
            pass
    
    return(nearby_venues)

In [11]:
chennai_venues = getNearbyVenues(names=df['Places'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adambakkam
Adyar
Alandur
Alapakkam
Alwarpet
Alwarthirunagar
Ambattur
Aminjikarai
Anna Nagar
Annanur
Arumbakkam
Ashok Nagar
Avadi
Ayanavaram
Besant Nagar
Basin Bridge
Chepauk
Chetput
Chintadripet
Chitlapakkam
Choolai
Choolaimedu
Chrompet
Egmore
Ennore
Foreshore Estate
Fort St. George
George Town
Gopalapuram
Government Estate
Guindy
Gerugambakkam
IIT Madras
Injambakkam
ICF
Iyyapanthangal
Jafferkhanpet
Karapakkam
Kattupakkam
K.K. Nagar
Keelkattalai
Kilpauk
Kodambakkam
Kodungaiyur
Kolathur
Korattur
Korukkupet
Kottivakkam
Kotturpuram
Kottur
Kovilambakkam
Koyambedu
Kundrathur
Madhavaram
Madhavaram Milk Colony
Madipakkam
Madambakkam
Maduravoyal
Manali
Manali New Town
Manapakkam
Mandaveli
Mangadu
Mannady
Mathur
Medavakkam
Meenambakkam
MGR Nagar
Minjur
Mogappair
MKB Nagar
Mount Road
Moolakadai
Moulivakkam
Mugalivakkam
Mudichur
Mylapore
Nandanam
Nanganallur
Nanmangalam
Neelankarai
Nemilichery
Nesapakkam
Nolambur
Noombal
Nungambakkam
Otteri
Padi
Pakkam
Palavakkam
Pallavaram
Pallikaranai
Pammal
Pa

In [12]:
liz = ['Gym','Bus','ATM','Pharmacy','Airport','Art','IT','Train','Book','Movie','Metro','Flower','Optical',\
       'Photo','Repair','Store','Cosmetics','Spa','Lounge','Rail','Multiplex','Park','Gift',\
      'Museum','Theater','Sport','Site','General','Market','Beach','Hall','Field','Daycare','Resort',\
      'Library','Lighthouse','Rental','Laundry','Travel','Cricket','Club','club','Car','Smoke','Record',\
      'Antique','Miscellaneous','Motor','Road','Observatory','Spot','Health','Soccer','Nature','Hostel',\
      'Church','River','Scenic','Platform','Bowling','Lake','Marina','Speak','Camp','Golf','Supermarket',\
      'Camp','Boutique','Arcade','Resort','Play','Plaza','Rest Area']

In [13]:
for word in liz:
    chennai_venues = chennai_venues[~chennai_venues["Venue Category"].str.contains(word)]

In [14]:
chennai_venues=chennai_venues.reset_index(drop=True)

In [15]:
import folium

# Creates map of Chennai using latitude and longitude values
map_chennai = folium.Map(location=[lat, long], zoom_start=10)

# Adds blue markers as venues
for lat, lng, venue in zip(chennai_venues['Venue Latitude'], chennai_venues['Venue Longitude'], chennai_venues["Venue"]):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai)  

# Adds red markers as places
for lat, lng, place in zip(df['Latitude'], df['Longitude'], df["Places"]):
    label = '{}'.format(place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai) 

map_chennai

In [16]:
                        ## randomly selecting one place and displaying the venues nearby ##

import random
from scipy.spatial import distance

plcin = random.randint(0,len(df)-1)

plc = df.loc[plcin,"Places"]
plc_lat = df.loc[plcin,"Latitude"]
plc_lng = df.loc[plcin,"Longitude"]

np_plc = np.array([plc_lat,plc_lng])
np_ven = chennai_venues[["Venue Latitude","Venue Longitude"]].values

for ind,value in enumerate(np_ven):
    np_ven[ind] = distance.euclidean(np_ven[ind],np_plc)
    
ind_sort = np.argsort(np_ven,axis=0)[0:5,0] #sort across rows and display top 5 venues

top5 = chennai_venues.loc[ind_sort[0:5],["Venue","Venue Latitude","Venue Longitude"]]#copying this to a dataframe

In [17]:
# Creates map of Chennai using latitude and longitude values
map_chennai = folium.Map(location=[lat, long], zoom_start=10)

# Adds blue markers as venues
for lat, lng, venue in zip(top5['Venue Latitude'], top5['Venue Longitude'], top5["Venue"]):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai)

#Add a red marker to indicate the location    
label = '{}'.format(plc)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [plc_lat, plc_lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai)

map_chennai